In [40]:
import json
import seaborn as sns
import csv
import pandas as pd
from glob import glob
import os
import re
from datetime import datetime, timezone

## nfts_traits_data_sorted

In [41]:
collection_slug="doodles-official" # [pudgypenguins,chimpersnft,doodles-official]
nfts_traits_data_path=f"./data/{collection_slug}/nfts_traits_data_sorted.json"
nfts_traits_data_csv_output_file = f'./data/{collection_slug}/nfts_traits_data_sorted.csv'

with open(nfts_traits_data_path, 'r') as f:
    data = json.load(f)

# Prepare CSV output

fieldnames = [
    'identifier', 'collection', 'contract', 'token_standard', 'name',
    'description', 'image_url', 'display_image_url', 'display_animation_url',
    'metadata_url', 'opensea_url', 'updated_at', 'is_disabled', 'is_nsfw',
    'animation_url', 'is_suspicious', 'creator', 
    # Trait columns will be added dynamically
    'owner_address', 'owner_quantity',
    'rarity_rank', 'rarity_score','rarity_tokens_scored'
]

# First pass to collect all possible trait types
trait_types = set()
for item in data:
    for trait in item['nft']['traits']:
        trait_types.add(trait['trait_type'])

# Add trait columns to fieldnames
for trait in sorted(trait_types):
    fieldnames.append(f'trait_{trait}')

# Write to CSV
with open(nfts_traits_data_csv_output_file, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for item in data:
        nft = item['nft']
        row = {
            'identifier': nft['identifier'],
            'collection': nft['collection'],
            'contract': nft['contract'],
            'token_standard': nft['token_standard'],
            'name': nft['name'],
            'description': nft['description'],
            'image_url': nft['image_url'],
            'display_image_url': nft['display_image_url'],
            'display_animation_url': nft['display_animation_url'],
            'metadata_url': nft['metadata_url'],
            'opensea_url': nft['opensea_url'],
            'updated_at': nft['updated_at'],
            'is_disabled': nft['is_disabled'],
            'is_nsfw': nft['is_nsfw'],
            'animation_url': nft['animation_url'],
            'is_suspicious': nft['is_suspicious'],
            'creator': nft['creator'],
            'owner_address': nft['owners'][0]['address'] if nft['owners'] else '',
            'owner_quantity': nft['owners'][0]['quantity'] if nft['owners'] else '',
            'rarity_rank': nft['rarity']['rank'] if nft['rarity'] else '',
            'rarity_score': nft['rarity']['score'] if nft['rarity'] else '',
            'rarity_tokens_scored': nft['rarity']['tokens_scored'] if nft['rarity'] else ''
            
        }
        
        # Add trait values
        traits = {t['trait_type']: t['value'] for t in nft['traits']}
        for trait in trait_types:
            row[f'trait_{trait}'] = traits.get(trait, '')
        
        writer.writerow(row)

print(f"CSV file saved as {nfts_traits_data_csv_output_file}")

CSV file saved as ./data/doodles-official/nfts_traits_data_sorted.csv


In [42]:
pd.read_csv(nfts_traits_data_csv_output_file).head()

,identifier,collection,contract,token_standard,name,description,image_url,display_image_url,display_animation_url,metadata_url,...,owner_quantity,rarity_rank,rarity_score,rarity_tokens_scored,trait_background,trait_body,trait_face,trait_hair,trait_head,trait_piercing
0,0,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #0,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmUEfFfwAh4wyB5UfHCVPUxis...,https://i.seadn.io/gcs/files/56a0c86cfb9300bd0...,NaN,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,1,7362,NaN,0,green,blue and yellow jacket,mustache,purple long,tan,NaN
1,1,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #1,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmTDxnzcvj2p3xBrKcGv1wxoy...,https://i.seadn.io/s/raw/files/3f3bd46d6f8dd02...,NaN,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,1,3322,NaN,0,grey,purple sweater with satchel,holographic beard,white bucket cap,gradient 2,NaN
2,2,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #2,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmbvZ2hbF3nEq5r3ijMEiSGss...,https://i.seadn.io/s/raw/files/e6740f79f6e4771...,NaN,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,1,4382,NaN,0,yellow,blue fleece,designer glasses,poopie,purple,NaN
3,3,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #3,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmVpwaCqLut3wqwB5KSQr2fGn...,https://i.seadn.io/s/raw/files/118b0d507cb0594...,NaN,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,1,5152,NaN,0,gradient 1,pink fleece,designer glasses,holographic mohawk,pale,NaN
4,4,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #4,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmcyuFVLbfBmSeQ9ynu4dk67r...,https://i.seadn.io/gcs/files/3956eb6cabdf3ff3f...,NaN,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,1,4123,NaN,0,gradient 2,spotted hoodie,happy,purple long,purple,NaN


## nfts_traits_event_data_sorted

In [59]:
collection_slug="doodles-official" #[pudgypenguins,chimpersnft,doodles-official]
nfts_traits_data_path=f"./data/{collection_slug}/nfts_traits_data_sorted.json"
nfts_sale_event_data_dir=f"./data/{collection_slug}/sale_events/"
nfts_traits_event_data_csv_output_file = f'./data/{collection_slug}/nfts_traits_event_data_sorted.csv'

with open(nfts_traits_data_path, 'r') as f:
    data = json.load(f)

# Prepare CSV output

fieldnames = [
    'identifier', 'collection', 'contract', 'token_standard', 'name',
    'description', 'image_url', 'display_image_url', 'display_animation_url',
    'metadata_url', 'opensea_url', 'updated_at', 'is_disabled', 'is_nsfw',
    'animation_url', 'is_suspicious', 'creator', 
    # Trait columns will be added dynamically
    'owner_address', 'owner_quantity',
    'rarity_rank', 'rarity_score','rarity_tokens_scored',
    'sale_event_number'
]

# First pass to collect all possible trait types
trait_types = set()
for item in data:
    for trait in item['nft']['traits']:
        trait_types.add(trait['trait_type'])

# Add trait columns to fieldnames
for trait in sorted(trait_types):
    fieldnames.append(f'trait_{trait}')

# Write to CSV
with open(nfts_traits_event_data_csv_output_file, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for item in data:
        nft = item['nft']
        row = {
            'identifier': nft['identifier'],
            'collection': nft['collection'],
            'contract': nft['contract'],
            'token_standard': nft['token_standard'],
            'name': nft['name'],
            'description': nft['description'],
            'image_url': nft['image_url'],
            'display_image_url': nft['display_image_url'],
            'display_animation_url': nft['display_animation_url'],
            'metadata_url': nft['metadata_url'],
            'opensea_url': nft['opensea_url'],
            'updated_at': nft['updated_at'],
            'is_disabled': nft['is_disabled'],
            'is_nsfw': nft['is_nsfw'],
            'animation_url': nft['animation_url'],
            'is_suspicious': nft['is_suspicious'],
            'creator': nft['creator'],
            'owner_address': nft['owners'][0]['address'] if nft['owners'] else '',
            'owner_quantity': nft['owners'][0]['quantity'] if nft['owners'] else '',
            'rarity_rank': nft['rarity']['rank'] if nft['rarity'] else '',
            'rarity_score': nft['rarity']['score'] if nft['rarity'] else '',
            'rarity_tokens_scored': nft['rarity']['tokens_scored'] if nft['rarity'] else ''
            
        }
        
        # Add trait values
        traits = {t['trait_type']: t['value'] for t in nft['traits']}
        for trait in trait_types:
            row[f'trait_{trait}'] = traits.get(trait, '')
        
        nft_sale_event_path=nfts_sale_event_data_dir+f"{collection_slug}_{nft['identifier']}.json"
        with open(nft_sale_event_path, 'r') as f:
            sale_event_data = json.load(f)
            row['sale_event_number'] = sale_event_data['metadata']['total_events']
        writer.writerow(row)

print(f"CSV file saved as {nfts_traits_event_data_csv_output_file}")

CSV file saved as ./data/doodles-official/nfts_traits_event_data_sorted.csv


In [60]:
pd.read_csv(nfts_traits_event_data_csv_output_file).head()

,identifier,collection,contract,token_standard,name,description,image_url,display_image_url,display_animation_url,metadata_url,...,rarity_rank,rarity_score,rarity_tokens_scored,sale_event_number,trait_background,trait_body,trait_face,trait_hair,trait_head,trait_piercing
0,0,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #0,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmUEfFfwAh4wyB5UfHCVPUxis...,https://i.seadn.io/gcs/files/56a0c86cfb9300bd0...,NaN,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,7362,NaN,0,2,green,blue and yellow jacket,mustache,purple long,tan,NaN
1,1,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #1,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmTDxnzcvj2p3xBrKcGv1wxoy...,https://i.seadn.io/s/raw/files/3f3bd46d6f8dd02...,NaN,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,3322,NaN,0,2,grey,purple sweater with satchel,holographic beard,white bucket cap,gradient 2,NaN
2,2,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #2,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmbvZ2hbF3nEq5r3ijMEiSGss...,https://i.seadn.io/s/raw/files/e6740f79f6e4771...,NaN,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,4382,NaN,0,4,yellow,blue fleece,designer glasses,poopie,purple,NaN
3,3,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #3,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmVpwaCqLut3wqwB5KSQr2fGn...,https://i.seadn.io/s/raw/files/118b0d507cb0594...,NaN,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,5152,NaN,0,1,gradient 1,pink fleece,designer glasses,holographic mohawk,pale,NaN
4,4,doodles-official,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,erc721,Doodle #4,A community-driven collectibles project featur...,https://ipfs.io/ipfs/QmcyuFVLbfBmSeQ9ynu4dk67r...,https://i.seadn.io/gcs/files/3956eb6cabdf3ff3f...,NaN,https://ipfs.io/ipfs/QmPMc4tcBsMqLRuCQtPmPe84b...,...,4123,NaN,0,1,gradient 2,spotted hoodie,happy,purple long,purple,NaN


## [all sale event] by nfts

In [61]:
# Get all JSON files and sort by identifier
def get_sorted_json_files(directory):
    files = []
    for json_file in glob(os.path.join(directory, '**/*.json'), recursive=True):
        # Extract identifier from filename (e.g., "pudgypenguins_123.json" -> 123)
        match = re.search(r'_(\d+)\.json$', json_file)
        if match:
            identifier = int(match.group(1))
            files.append((identifier, json_file))
    
    # Sort by identifier numerically
    files.sort(key=lambda x: x[0])
    return [file[1] for file in files]

def convert_unix_to_iso(timestamp):
    """Convert Unix timestamp to ISO 8601 format string"""
    if timestamp and str(timestamp).isdigit():
        return datetime.fromtimestamp(timestamp, tz=timezone.utc).isoformat() + 'Z'
    return ''


In [62]:

collection_slug="doodles-official" #[pudgypenguins,chimpersnft,doodles-official]
nfts_sale_event_data_dir=f"./data/{collection_slug}/sale_events/"
collection_event_data_csv_output_file = f'./data/{collection_slug}/collection_event_data.csv'
eth_precision=18 # save ETH amount with 6 decimal precision after converting from Wei

json_files = get_sorted_json_files(nfts_sale_event_data_dir)
total_files = len(json_files)

# Prepare CSV fields
fieldnames = [
    # Metadata fields
    # 'collection_slug', 'contract', 'identifier', 'total_events',
    'collection_slug', 'identifier',
    
    # Event fields
    'event_type', 'order_hash', 'chain', 'protocol_address', 
    'closing_date', 'closing_date_iso',
    'quantity', 'seller', 'buyer',
    'payment_quantity', 'payment_token', 'payment_decimals', 'payment_symbol','payment_symbol_amount',
    'transaction_hash', 'event_timestamp', 'event_timestamp_iso',
    
    # NFT fields
    # 'nft_name', 'nft_description', 'nft_image_url', 'nft_display_image_url',
    # 'nft_metadata_url', 'nft_opensea_url', 'nft_updated_at',
    # 'nft_token_standard', 'nft_is_disabled', 'nft_is_nsfw'
]

with open(collection_event_data_csv_output_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for i, json_file in enumerate(json_files, 1):
        # try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                metadata = data.get('metadata', {})
                
                # Progress update
                if i % 100 == 0 or i == total_files:
                    print(f"Processing {i}/{total_files} - ID: {metadata.get('identifier')}")
                # if data.get('events', []) == []:
                #     row = {
                #             # Metadata
                #             'collection_slug': metadata.get('collection_slug'),
                #             'identifier': metadata.get('identifier'),
                            
                #             # Event data
                #             'event_type': None,
                #             'order_hash': None,
                #             'chain': None,
                #             'protocol_address': None,
                #             'closing_date': None,
                #             'closing_date_iso': None,
                #             'quantity': None,
                #             'seller': None,
                #             'buyer': None,
                #             'payment_quantity': None,
                #             'payment_token': None,
                #             'payment_decimals': None,
                #             'payment_symbol': None,
                #             'payment_symbol_amount': None,
                #             'transaction_hash': None,
                #             'event_timestamp': None,
                #             'event_timestamp_iso': None,
                        
                #         }
                # else:
                for event in data.get('events', []):
                    nft = event.get('nft', {})
                    payment = event.get('payment', {})
                    if not payment:
                        continue
                    # Convert timestamps
                    closing_date_unix = event.get('closing_date')
                    event_timestamp_unix = event.get('event_timestamp')
                    
                    # Convert quantity to ETH amount
                    payment_symbol_amount = round(event.get('quantity') * (float(payment.get('quantity')) / 10**payment.get('decimals')),eth_precision)
                    
                    row = {
                        # Metadata
                        'collection_slug': metadata.get('collection_slug'),
                        'identifier': metadata.get('identifier'),
                        
                        # Event data
                        'event_type': event.get('event_type'),
                        'order_hash': event.get('order_hash'),
                        'chain': event.get('chain'),
                        'protocol_address': event.get('protocol_address'),
                        'closing_date': closing_date_unix,
                        'closing_date_iso': convert_unix_to_iso(closing_date_unix),
                        'quantity': event.get('quantity'),
                        'seller': event.get('seller'),
                        'buyer': event.get('buyer'),
                        'payment_quantity': payment.get('quantity'),
                        'payment_token': payment.get('token_address'),
                        'payment_decimals': payment.get('decimals'),
                        'payment_symbol': payment.get('symbol'),
                        'payment_symbol_amount': payment_symbol_amount,
                        'transaction_hash': event.get('transaction'),
                        'event_timestamp': event_timestamp_unix,
                        'event_timestamp_iso': convert_unix_to_iso(event_timestamp_unix),
                    
                    }
                    
                    writer.writerow(row)
                
        # except Exception as e:
        #     print(f"Error in {os.path.basename(json_file)}: {str(e)}")

Processing 100/10000 - ID: 99
Processing 200/10000 - ID: 199
Processing 300/10000 - ID: 299
Processing 400/10000 - ID: 399
Processing 500/10000 - ID: 499
Processing 600/10000 - ID: 599
Processing 700/10000 - ID: 699
Processing 800/10000 - ID: 799
Processing 900/10000 - ID: 899
Processing 1000/10000 - ID: 999
Processing 1100/10000 - ID: 1099
Processing 1200/10000 - ID: 1199
Processing 1300/10000 - ID: 1299
Processing 1400/10000 - ID: 1399
Processing 1500/10000 - ID: 1499
Processing 1600/10000 - ID: 1599
Processing 1700/10000 - ID: 1699
Processing 1800/10000 - ID: 1799
Processing 1900/10000 - ID: 1899
Processing 2000/10000 - ID: 1999
Processing 2100/10000 - ID: 2099
Processing 2200/10000 - ID: 2199
Processing 2300/10000 - ID: 2299
Processing 2400/10000 - ID: 2399
Processing 2500/10000 - ID: 2499
Processing 2600/10000 - ID: 2599
Processing 2700/10000 - ID: 2699
Processing 2800/10000 - ID: 2799
Processing 2900/10000 - ID: 2899
Processing 3000/10000 - ID: 2999
Processing 3100/10000 - ID: 30

In [63]:
len(json_files)

10000

In [64]:
collection_event_data=pd.read_csv(collection_event_data_csv_output_file)
collection_event_data.head()

,collection_slug,identifier,event_type,order_hash,chain,protocol_address,closing_date,closing_date_iso,quantity,seller,buyer,payment_quantity,payment_token,payment_decimals,payment_symbol,payment_symbol_amount,transaction_hash,event_timestamp,event_timestamp_iso
0,doodles-official,0,sale,NaN,ethereum,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,1634663005,2021-10-19T17:03:25+00:00Z,1,0xeff53613068a15a6845943e447fca03936c283c8,0xad097fdcd58535250c59807d6683e0a6b688d6cc,10000000000000000000,0x0000000000000000000000000000000000000000,18,ETH,10.00,0x663e8b5d7bd8871fba2b1fc969de039c0169e6e29222...,1634663005,2021-10-19T17:03:25+00:00Z
1,doodles-official,0,sale,NaN,ethereum,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,1634476925,2021-10-17T13:22:05+00:00Z,1,0x47024c554fac0863391dfa6fc36deda9e31c70cf,0xeff53613068a15a6845943e447fca03936c283c8,5000000000000000000,0x0000000000000000000000000000000000000000,18,ETH,5.00,0xe3f31ec4f8404901c0c09a7bc2c49d1a3003bc0d5477...,1634476925,2021-10-17T13:22:05+00:00Z
2,doodles-official,1,sale,NaN,ethereum,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,1636782106,2021-11-13T05:41:46+00:00Z,1,0x8b078d0cc0f3b67f18c0877cc20519b04a427abe,0x407898b364cd540fa83f8fcfcf591270cd4fa76a,10000000000000000000,0x0000000000000000000000000000000000000000,18,ETH,10.00,0xc158d7cf1d459dc043dd3752eb1079a9d8d799b7b9f1...,1636782106,2021-11-13T05:41:46+00:00Z
3,doodles-official,1,sale,NaN,ethereum,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,1634476623,2021-10-17T13:17:03+00:00Z,1,0x47024c554fac0863391dfa6fc36deda9e31c70cf,0x8b078d0cc0f3b67f18c0877cc20519b04a427abe,1990000000000000000,0x0000000000000000000000000000000000000000,18,ETH,1.99,0xdd59aceac3a12ffefb4f6d9c12823846f56070bd58be...,1634476623,2021-10-17T13:17:03+00:00Z
4,doodles-official,2,sale,NaN,ethereum,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,1645223264,2022-02-18T22:27:44+00:00Z,1,0xcaf5ad200387579912549b499326d3f536aa406f,0xab3e223534a10ba14f5abb51577f30c2a1ccb9f7,22220000000000000000,0x0000000000000000000000000000000000000000,18,ETH,22.22,0x0782a1158f1b46d02bd49ad07a8d8f623007c45f9975...,1645223264,2022-02-18T22:27:44+00:00Z


In [66]:
print(collection_event_data['payment_symbol'].unique())
collection_event_data['payment_symbol'].value_counts()

['ETH' 'WETH']


payment_symbol
ETH     41843
WETH    39448
Name: count, dtype: int64

## [stats of all sale event] by nfts

In [67]:
collection_slug

'doodles-official'

In [68]:
'''
collection_event_nft_stats
'''
collection_event_nft_stats_pth=f"./data/{collection_slug}/collection_event_nft_stats.csv"
nfts_traits_event_data_csv_output_file = f'./data/{collection_slug}/nfts_traits_event_data_sorted.csv'
collection_event_data_csv_output_file = f'./data/{collection_slug}/collection_event_data.csv'

nfts_traits_event_data= pd.read_csv(nfts_traits_event_data_csv_output_file)
collection_event_data=pd.read_csv(collection_event_data_csv_output_file)

all_identifiers = nfts_traits_event_data[['identifier']].drop_duplicates()

## filter out the events using eth or weth
collection_event_data_filter = collection_event_data[
    (collection_event_data['payment_symbol'].str.lower() == 'eth') |
    (collection_event_data['payment_symbol'].str.lower() == 'weth')
]


stats_with_trades=collection_event_data_filter.groupby('identifier').agg(
    avg_price_eth=('payment_symbol_amount', lambda x: x.mean()),
    trade_count=('payment_symbol_amount', 'count'),
    first_sale=('closing_date_iso', 'min'),
    last_sale=('closing_date_iso', 'max'),
    total_volume=('payment_quantity', lambda x: x.astype(float).sum()),
    total_volume_eth=('payment_symbol_amount', lambda x: x.sum()),
    lower_price=('payment_symbol_amount', lambda x: x.min()),
    higher_price=('payment_symbol_amount', lambda x: x.max()),
    
    
).reset_index()


collection_event_nft_stats= pd.merge(
    all_identifiers,
    stats_with_trades,
    on='identifier',
    how='left'
)

fill_values = {
    'avg_price_eth': 0,
    'trade_count': 0,
    'first_sale': pd.NaT,  # 无交易日期
    'last_sale': pd.NaT,
    'total_volume': 0,
    'total_volume_eth': 0,
    'lower_price': 0,
    'higher_price': 0
}
collection_event_nft_stats.fillna(fill_values, inplace=True)


rarity_rank_mapping = nfts_traits_event_data[['identifier', 'rarity_rank']].drop_duplicates()


# 合并稀有度数据（注意处理可能的缺失值）
collection_event_nft_stats = pd.merge(
    collection_event_nft_stats,
    rarity_rank_mapping,
    on='identifier',
    how='left'
)

collection_event_nft_stats['has_trades'] = collection_event_nft_stats['trade_count'] > 0
collection_event_nft_stats.to_csv(collection_event_nft_stats_pth, index=False)
collection_event_nft_stats

,identifier,avg_price_eth,trade_count,first_sale,last_sale,total_volume,total_volume_eth,lower_price,higher_price,rarity_rank,has_trades
0,0,7.500000,2.0,2021-10-17T13:22:05+00:00Z,2021-10-19T17:03:25+00:00Z,1.500000e+19,15.0000,5.0000,10.000,7362,True
1,1,5.995000,2.0,2021-10-17T13:17:03+00:00Z,2021-11-13T05:41:46+00:00Z,1.199000e+19,11.9900,1.9900,10.000,3322,True
2,2,11.640000,4.0,2021-10-17T13:18:27+00:00Z,2022-02-18T22:27:44+00:00Z,4.656000e+19,46.5600,2.0000,22.220,4382,True
3,3,18.880000,1.0,2022-01-05T02:58:00+00:00Z,2022-01-05T02:58:00+00:00Z,1.888000e+19,18.8800,18.8800,18.880,5152,True
4,4,1.200000,1.0,2021-10-17T13:11:34+00:00Z,2021-10-17T13:11:34+00:00Z,1.200000e+18,1.2000,1.2000,1.200,4123,True
...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,1.399000,2.0,2021-10-18T21:58:36+00:00Z,2021-10-23T19:09:33+00:00Z,2.798000e+18,2.7980,0.8490,1.949,9768,True
9996,9996,3.116900,10.0,2021-10-20T00:35:20+00:00Z,2024-01-10T15:00:47+00:00Z,3.116900e+19,31.1690,0.8890,14.100,9253,True
9997,9997,29.896667,3.0,2021-10-20T06:04:31+00:00Z,2022-05-03T05:27:48+00:00Z,8.969000e+19,89.6900,3.6900,50.000,330,True
9998,9998,5.353025,4.0,2021-11-17T22:29:31+00:00Z,2022-02-06T20:32:28+00:00Z,2.141210e+19,21.4121,1.9321,13.700,7698,True
